# Install spaCy

pip install -U pip setuptools wheel
pip install -U spacy
python -m spacy download en_core_web_md

# Imports

In [1]:
from time import time

import numpy as np
import pandas as pd

import spacy
from spacy.tokens import Doc

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, classification_report

# Чтение и анализ датасета

In [2]:
df = pd.read_csv('IMDB Dataset.csv')
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


Дата сет представляет собой 50 000 пользовательских отзывов на 
фильмы с классификацией положительный `(positive)` или отрицательный `(negative)`

Посмотрим на таргет

In [3]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

Таргет сбалансирован, имеем по 25000 положительных и отрицательных отзывов

# Формирование выборки. Векторизация текста.  

Выполняя это задания я провел ряд экспериментов и выяснил, что spaCy 
будет обрабатывать весь датасет не менее 30 минут (на моем компьютере)

Поэтому, для скорости, дальнейшую работу будем выполнять над частью 
датасета в 5000 отзывов 

In [4]:
QTY = 5000

In [5]:
X_str = df['review'][:QTY]
y = df['sentiment'][:QTY].replace({'negative': 0, 'positive': 1})

Загрузим языковой конвейер `en_core_web_md`

In [6]:
nlp = spacy.load('en_core_web_md')

Сформируем датасет путем обработки текстовых записей (обзоров на фильмы) языковым конвейером.

In [7]:
start = time()

X_doc = X_str.apply(nlp)

print(f'Time: {time() - start:.0f} sec')

Time: 173 sec


In [8]:
X_doc

0       (One, of, the, other, reviewers, has, mentione...
1       (A, wonderful, little, production, ., <, br, /...
2       (I, thought, this, was, a, wonderful, way, to,...
3       (Basically, there, 's, a, family, where, a, li...
4       (Petter, Mattei, 's, ", Love, in, the, Time, o...
                              ...                        
4995    (An, interesting, slasher, film, with, multipl...
4996    (i, watched, this, series, when, it, first, ca...
4997    (Once, again, Jet, Li, brings, his, charismati...
4998    (I, rented, this, movie, ,, after, hearing, Ch...
4999    (This, was, a, big, disappointment, for, me, ....
Name: review, Length: 5000, dtype: object

In [9]:
print(type(X_doc[0]))
dir(X_doc[0])

<class 'spacy.tokens.doc.Doc'>


['_',
 '__bytes__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '_bulk_merge',
 '_context',
 '_get_array_attrs',
 '_realloc',
 '_vector',
 '_vector_norm',
 'cats',
 'char_span',
 'copy',
 'count_by',
 'doc',
 'ents',
 'extend_tensor',
 'from_array',
 'from_bytes',
 'from_dict',
 'from_disk',
 'from_docs',
 'from_json',
 'get_extension',
 'get_lca_matrix',
 'has_annotation',
 'has_extension',
 'has_unknown_spaces',
 'has_vector',
 'is_nered',
 'is_parsed',
 'is_sentenced',
 'is_tagged',
 'lang',
 'lang_',
 'mem',
 'noun_chunks',
 'noun_chunks_iterator',
 'remove_extension',
 'retokenize',
 'sentiment',
 'sents',
 'set

Языковой конвейер возвращает объект типа `spacy.tokens.doc.Doc`
У этого объекта есть много атрибутов, один из которых - `vector`.
Это векторное представление исходного текста.

Эти вектора мы и будем использовать для обучения модели.

Сформируем датасет с которым в дальнейшем будет работать наша модель,
используя векторное представление исходных текстов (обзоров). 

In [10]:
X = np.zeros((1, 300))  # инициализируем матрицу векторов нулевым вектором-строкой
for doc in X_doc:
    vector = doc.vector.reshape(1, -1)
    X = np.vstack((X, vector))
X = np.delete(arr=X, obj=0, axis=0)  # удаляем первую нулевую строку в полученной матрице

In [11]:
X

array([[-1.75048316,  0.70223123, -2.26069546, ..., -0.75643164,
        -2.84037662,  1.16227973],
       [-1.72586012,  0.40414372, -0.70506632, ..., -1.227211  ,
        -3.30375242,  0.76335639],
       [-2.07742572,  1.27698457, -1.43668723, ...,  0.01261111,
        -3.35728073,  1.43638575],
       ...,
       [-1.26115525,  1.3370682 , -2.53299332, ..., -0.9334181 ,
        -3.5286839 ,  1.42038274],
       [-1.79231286,  0.87925476, -2.40937471, ..., -0.49132967,
        -3.41382909,  0.77604485],
       [-1.44713509,  1.85639131, -2.43208957, ...,  0.91516876,
        -3.60654378,  1.49083591]])

Перед тем, как начать обучение модели еще раз проверим, что наш таргет репрезентативен.

In [12]:
y.value_counts()

sentiment
0    2532
1    2468
Name: count, dtype: int64

Все в порядке. У нас есть примерно поровну положительных и отрицательных отзывов.

# Обучение и проверка модели классификации

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [14]:
model = RandomForestClassifier(random_state=42)

model_cv_score = np.mean(
    cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy", n_jobs=-1)
)
print(f"Model cross validation score: {model_cv_score}\n")

Model cross validation score: 0.7254285714285714


In [15]:
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [16]:
y_predicted = model.predict(X_test)
model_test_score = accuracy_score(y_true=y_test, y_pred=y_predicted)

print(f"Model test set score: {model_test_score}")
print("\n", classification_report(y_true=y_test, y_pred=y_predicted))

Model test set score: 0.7273333333333334

               precision    recall  f1-score   support

           0       0.75      0.72      0.73       783
           1       0.71      0.74      0.72       717

    accuracy                           0.73      1500
   macro avg       0.73      0.73      0.73      1500
weighted avg       0.73      0.73      0.73      1500


На тестовой выборке наша модель показала точность 72.7%
Это не мало, но и не сказать, что много.

Попробуем что-то с этим сделать.

# Обучение модели только на прилагательных

Напомню, что речь о классификации отзывов на фильмы на положительные и отрицательные.
Как правило, для того, чтобы выразить свое отношение к чему-либо, 
мы используем прилагательные, например: `хороший`, `плохой` и т.д.

Идея состоит в том, чтобы выделить из отзывов только прилагательные и обучить 
модель только на них, исключив, таким образом, влияние остальных слов.

Формируем датасет из прилагательных

In [17]:
from spacy.tokens import Doc

def get_adjectives(doc: spacy.tokens.Doc) -> spacy.tokens.Doc:
    adjectives = [token.lemma_ for token in doc if token.pos_ == "ADJ"]
    return Doc(nlp.vocab, words=adjectives)

X_adj_doc = X_doc.apply(get_adjectives)
X_adj_doc

0       (other, right, first, faint, hearted, timid, h...
1       (wonderful, little, unassuming-, old, entire, ...
2       (wonderful, hot, light, hearted, simplistic, w...
3       (little, slow, watchable, divorcing, real, sim...
4       (stunning, vivid, human, different, same, pres...
                              ...                        
4995    (interesting, multiple, typical, creepy, unusu...
4996    (old, good, weekly, reverential, broad, diffic...
4997    (charismatic, super, normal, peaceful, other, ...
4998    (other, least, main, much, raw, bare, unbiased...
4999                            (big, bad, hard, typical)
Name: review, Length: 5000, dtype: object

Переходим к векторам

In [18]:
X = np.zeros((1, 300))  # инициализируем матрицу векторов нулевым вектором-строкой
for doc in X_adj_doc:
    vector = doc.vector.reshape(1, -1)
    X = np.vstack((X, vector))
X = np.delete(arr=X, obj=0, axis=0)  # удаляем первую нулевую строку в полученной матрице

In [19]:
X

array([[-0.40639281,  0.23371167, -1.50917244, ..., -0.14155047,
        -3.28597641,  0.88043147],
       [-0.42534426,  0.08912002, -2.53276205, ...,  1.69812787,
        -3.43252563,  0.15116785],
       [ 0.18922471,  0.26509333, -0.47760415, ...,  0.9715609 ,
        -4.76701069,  1.451056  ],
       ...,
       [-0.84462392, -0.20629264, -2.06435227, ..., -0.55455095,
        -3.80893469,  1.20794415],
       [-0.15281355, -0.78035438, -1.13539529, ...,  0.04319851,
        -4.12046623,  0.79743236],
       [ 1.75609982,  1.4727999 , -3.27535248, ...,  2.70974994,
        -2.8594625 ,  2.65460014]])

Обучение модели, вывод метрик

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model.fit(X_train, y_train)

y_predicted = model.predict(X_test)
model_test_score = accuracy_score(y_true=y_test, y_pred=y_predicted)

print(f"Model test set score: {model_test_score}")
print("\n", classification_report(y_true=y_test, y_pred=y_predicted))

Model test set score: 0.78

               precision    recall  f1-score   support

           0       0.81      0.76      0.78       783
           1       0.75      0.80      0.78       717

    accuracy                           0.78      1500
   macro avg       0.78      0.78      0.78      1500
weighted avg       0.78      0.78      0.78      1500


Как мы видим, точность модели увеличилась до 78%